## POLICY GRADIENT on CartPole

Policy Gradient algorithms find an optimal behavior strategy optimizing directly the policy. 
The policy is a parametrized function respect to $\theta$ $\pi_\theta(a|s)$

The reward function is defined as 
$$J(\theta) = \sum_{s}d^\pi(s)\sum_{a}\pi_\theta(a|s)Q^\pi(s,a)$$

In Vanilla Policy Gradient, we estimate the return $R_t$ (REINFORCE algorithm) and update the policy subtracting a baseline value from $R_t$ to reduce the variance.

<img src="imgs/Vanilla_policy_gradient.png" alt="drawing" width="500"/>
Credit: John Schulman

In [ ]:
import numpy as np
import gym
from tensorboardX import SummaryWriter

import time
from collections import namedtuple
from collections import deque
import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class PG_nn(nn.Module):
    '''
    Policy neural net
    '''
    def __init__(self, input_shape, n_actions):
        super(PG_nn, self).__init__()

        self.mlp = nn.Sequential(
            nn.Linear(input_shape[0], 64),
            nn.ReLU(),
            nn.Linear(64, n_actions))

    def forward(self, x):
        return self.mlp(x.float())

In [ ]:
def discounted_rewards(memories, gamma):
    '''
    Compute the discounted reward backward
    '''

    disc_rew = np.zeros(len(memories))
    run_add = 0

    for t in reversed(range(len(memories))):
        if memories[t].done: run_add = 0
        run_add = run_add * gamma + memories[t].reward
        disc_rew[t] = run_add

    return disc_rew

In [ ]:
Memory = namedtuple('Memory', ['obs', 'action', 'new_obs', 'reward', 'done'], verbose=False, rename=False)

GAMMA = 0.99
LEARNING_RATE = 0.002
ENTROPY_BETA = 0.01
ENV_NAME = 'CartPole-v0'

MAX_N_GAMES = 10000
n_games = 0

device = 'cpu'

now = datetime.datetime.now()
date_time = "{}_{}.{}.{}".format(now.day, now.hour, now.minute, now.second)

In [ ]:
env = gym.make(ENV_NAME)
obs = env.reset()

# Initialize the writer
writer = SummaryWriter(log_dir='content/runs/A2C'+ENV_NAME+'_'+date_time)

# create the agent neural net
action_n = env.action_space.n
agent_nn = PG_nn(env.observation_space.shape, action_n).to(device)

# Adam optimizer
optimizer = optim.Adam(agent_nn.parameters(), lr=LEARNING_RATE)

experience = []
tot_reward = 0
n_iter = 0
# deque list to keep the baseline
baseline = deque(maxlen=30000)
game_rew = 0

## MAIN BODY
while n_games < MAX_N_GAMES:

    n_iter += 1

    # execute the agent
    act = agent_nn(torch.tensor(obs))
    act_soft = F.softmax(act)
    # get an action following the policy distribution
    action = int(np.random.choice(np.arange(action_n), p=act_soft.detach().numpy(), size=1))

    # make a step in the env
    new_obs, reward, done, _ = env.step(action)

    game_rew += reward
    # update the experience list with the last memory
    experience.append(Memory(obs=obs, action=action, new_obs=new_obs, reward=reward, done=done))

    obs = new_obs

    if done:
        # Calculate the discounted rewards
        disc_rewards = discounted_rewards(experience, GAMMA)

        # update the baseline
        baseline.extend(disc_rewards)
        # subtract the baseline mean from the discounted reward.
        disc_rewards -= np.mean(baseline)

        # run the agent NN on the obs in the experience list
        acts = agent_nn(torch.tensor([e.obs for e in experience]))

        # take the log softmax of the action taken previously
        game_act_log_softmax_t = F.log_softmax(acts, dim=1)[:,[e.action for e in experience]]

        disc_rewards_t = torch.tensor(disc_rewards, dtype=torch.float32).to(device)

        # compute the loss entropy
        l_entropy = ENTROPY_BETA * torch.mean(torch.sum(F.softmax(acts, dim=1) * F.log_softmax(acts, dim=1), dim=1))

        # compute the loss
        loss = - torch.mean(disc_rewards_t * game_act_log_softmax_t)
        loss = loss + l_entropy

        # optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print the stats
        writer.add_scalar('loss', loss, n_iter)
        writer.add_scalar('reward', game_rew, n_iter)

        print(n_games, loss.detach().numpy(), game_rew, np.mean(disc_rewards), np.mean(baseline))

        # reset the variables and env
        experience = []
        game_rew = 0
        obs = env.reset()
        n_games += 1


writer.close()

![Reward](imgs/reward_pg.png)